In [1]:
# @Time     : Jan. 02, 2019 22:17
# @Author   : Veritas YIN
# @FileName : main.py
# @Version  : 1.0
# @Project  : Orion
# @IDE      : PyCharm
# @Github   : https://github.com/VeritasYin/Project_Orion

import os

os.environ["CUDA_VISIBLE_DEVICES"] = "0"
from os.path import join as pjoin

import tensorflow as tf

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
tf.Session(config=config)

from utils.math_graph import *
from data_loader.data_utils import *
from models.trainer import model_train
from models.tester import model_test

import argparse

parser = argparse.ArgumentParser()
parser.add_argument('--n_route', type=int, default=228)
parser.add_argument('--n_his', type=int, default=12)
parser.add_argument('--n_pred', type=int, default=9)
parser.add_argument('--batch_size', type=int, default=50)
parser.add_argument('--epoch', type=int, default=50)
parser.add_argument('--save', type=int, default=10)
parser.add_argument('--ks', type=int, default=3)
parser.add_argument('--kt', type=int, default=3)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--opt', type=str, default='RMSProp')
parser.add_argument('--graph', type=str, default='default')
parser.add_argument('--inf_mode', type=str, default='merge')
parser.add_argument("-f", type=str, default="")

args = parser.parse_args()
print(f'Training configs: {args}')

n, n_his, n_pred = args.n_route, args.n_his, args.n_pred
Ks, Kt = args.ks, args.kt
# blocks: settings of channel size in st_conv_blocks / bottleneck design
blocks = [[1, 32, 64], [64, 32, 128]]

# Load wighted adjacency matrix W
if args.graph == 'default':
    #W = weight_matrix(pjoin('./dataset', f'PeMSD7_W_{n}.csv'))
    W = weight_matrix(pjoin('./dataset', f'W_{n}.csv'))
else:
    # load customized graph weight matrix
    W = weight_matrix(pjoin('./dataset', args.graph))

# Calculate graph kernel
L = scaled_laplacian(W)
# Alternative approximation method: 1st approx - first_approx(W, n).
Lk = cheb_poly_approx(L, Ks, n)
tf.add_to_collection(name='graph_kernel', value=tf.cast(tf.constant(Lk), tf.float32))

# Data Preprocessing
#data_file = f'PeMSD7_V_{n}.csv'
data_file = f'V_{n}.csv'
n_train, n_val, n_test = 34, 5, 5
#PeMS = data_gen(pjoin('./dataset', data_file), (n_train, n_val, n_test), n, n_his + n_pred)
PeMS = data_gen_v2(pjoin('./dataset', data_file), (n_train, n_val, n_test), n, n_his + n_pred)

/home/rodr/code/env_oldtf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/rodr/code/env_oldtf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/rodr/code/env_oldtf/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/rodr/code/env_oldtf/lib/python3.7/site-packages/tensorflow/python/f

Training configs: Namespace(batch_size=50, epoch=50, f='/home/rodr/.local/share/jupyter/runtime/kernel-946bb1a1-9e4d-45c7-bb12-b9a8b8299d7c.json', graph='default', inf_mode='merge', ks=3, kt=3, lr=0.001, n_his=12, n_pred=9, n_route=228, opt='RMSProp', save=10)
Residuals of Historical Average (on all data) -> MAPE: 0.090; MAE:  3.526; RMSE: 6.439
(9112, 21, 228, 1)
(1340, 21, 228, 1)
(1340, 21, 228, 1)
(9112, 21, 228, 1)
(1340, 21, 228, 1)
(1340, 21, 228, 1)


In [2]:
PeMS.get_data("train").mean()

1.719624500902554e-16

In [3]:
model_train(PeMS, blocks, args)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
Epoch  0, Step   0: [215857.656, 27727.828]
Epoch  0, Step  50: [75308.734, 23614.428]
Epoch  0, Step 100: [79658.562, 20904.184]
Epoch  0, Step 150: [65176.652, 25246.775]
Epoch  0 Training Time 49.110s
Time Step 3: MAPE  7.968%,  7.181%; MAE  2.952, 2.845; RMSE  5.599,  5.294.
Time Step 6: MAPE  9.407%,  8.687%; MAE  3.576, 3.481; RMSE  6.402,  6.186.
Time Step 9: MAPE 10.389%,  9.729%; MAE  4.019, 3.947; RMSE  6.929,  6.771.
Epoch  0 Inference Time 49.512s
Epoch  1, Step   0: [53540.703, 27395.816]
Epoch  1, Step  50: [41804.477, 24569.859]
Epoch  1, Step 100: [38625.336, 25639.309]
Epoch  1, Step 150: [36742.266, 27220.883]
Epoch  1 Training Time 48.493s
Time Step 3: MAPE  7.224%,  6.597%; MAE  2.766, 2.696; RMSE  5.107,  4.879.
Time Step 6: MAPE  9.2

Epoch 16, Step   0: [20118.562, 24567.686]
Epoch 16, Step  50: [22595.730, 26758.023]
Epoch 16, Step 100: [22146.945, 26236.578]
Epoch 16, Step 150: [20025.078, 24636.596]
Epoch 16 Training Time 48.271s
Time Step 3: MAPE  6.273%,  5.914%; MAE  2.469, 2.434; RMSE  4.420,  4.293.
Time Step 6: MAPE  8.280%,  7.696%; MAE  3.159, 3.081; RMSE  5.816,  5.597.
Time Step 9: MAPE  9.219%,  8.573%; MAE  3.530, 3.411; RMSE  6.426,  6.186.
Epoch 16 Inference Time 49.095s
Epoch 17, Step   0: [18797.822, 23035.115]
Epoch 17, Step  50: [20174.420, 25238.500]
Epoch 17, Step 100: [18636.764, 23607.340]
Epoch 17, Step 150: [19384.350, 24056.693]
Epoch 17 Training Time 48.551s
Time Step 3: MAPE  6.264%,  5.828%; MAE  2.461, 2.419; RMSE  4.402,  4.270.
Time Step 6: MAPE  8.272%,  7.637%; MAE  3.159, 3.104; RMSE  5.816,  5.618.
Time Step 9: MAPE  9.219%,  8.660%; MAE  3.530, 3.508; RMSE  6.426,  6.340.
Epoch 17 Inference Time 48.615s
Epoch 18, Step   0: [22727.930, 27652.449]
Epoch 18, Step  50: [20723.461,

Epoch 34, Step   0: [18030.506, 23321.680]
Epoch 34, Step  50: [19320.730, 24869.164]
Epoch 34, Step 100: [21326.826, 27918.480]
Epoch 34, Step 150: [18547.057, 23217.703]
Epoch 34 Training Time 47.959s
Time Step 3: MAPE  6.080%,  5.726%; MAE  2.436, 2.405; RMSE  4.310,  4.213.
Time Step 6: MAPE  8.126%,  7.607%; MAE  3.128, 3.117; RMSE  5.703,  5.635.
Time Step 9: MAPE  9.112%,  8.751%; MAE  3.500, 3.564; RMSE  6.344,  6.456.
Epoch 34 Inference Time 49.169s
Epoch 35, Step   0: [20759.332, 25988.168]
Epoch 35, Step  50: [17631.473, 22802.576]
Epoch 35, Step 100: [18022.496, 24998.330]
Epoch 35, Step 150: [19680.826, 25476.480]
Epoch 35 Training Time 48.353s
Time Step 3: MAPE  6.080%,  5.726%; MAE  2.436, 2.405; RMSE  4.310,  4.213.
Time Step 6: MAPE  8.126%,  7.607%; MAE  3.128, 3.117; RMSE  5.703,  5.635.
Time Step 9: MAPE  9.112%,  8.751%; MAE  3.500, 3.564; RMSE  6.344,  6.456.
Epoch 35 Inference Time 24.377s
Epoch 36, Step   0: [19367.363, 24738.348]
Epoch 36, Step  50: [18693.732,

In [4]:
model_test(PeMS, PeMS.get_len('test'), n_his, n_pred, args.inf_mode)

Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ./output/models/STGCN-9150
>> Loading saved model from ./output/models/STGCN-9150 ...
Time Step 3: MAPE  5.748%; MAE  2.412; RMSE  4.211.
Time Step 6: MAPE  7.569%; MAE  3.097; RMSE  5.576.
Time Step 9: MAPE  8.597%; MAE  3.496; RMSE  6.294.
Model Test Time 29.880s
Testing model finished!
